In [1]:
# pip3 install mariadb
# pip3 install pandas
# pip3 install numpy  
# sudo apt-get install libatlas-base-dev
# pip3 install matplotlib
# pip3 install ipympl
# pip3 install ipywidgets

import os
import sys
import mariadb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, interactive_output, fixed, interact_manual, IntSlider, IntRangeSlider, HBox
import ipywidgets as widgets

In [2]:
username = os.environ.get("db_user")
password = os.environ.get("db_pwd")

w_db = widgets.RadioButtons(
        options=['reformer', 'catalyst'],
        description='Select Database:',
        disabled=False
        )
display(w_db)
db_name = w_db.value

RadioButtons(description='Select Database:', options=('reformer', 'catalyst'), value='reformer')

In [3]:
try:
    # Connect to MariaDB Platform
    conn = mariadb.connect(
        user=username,
        password=password,
        host="localhost",
        port=3306,
        database=db_name,
        autocommit=True
    )

    # Get Cursor for tx
    cur = conn.cursor(named_tuple=False)
    #print(conn)
    print(cur)
    print(f"Connected to {db_name}@localhost ")
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

Connected to reformer@localhost 


In [4]:
cur.execute('SHOW tables')
table_dropdown = widgets.Dropdown(options=[u for i in cur.fetchall()[:] for u in i][::-1])
cur.execute(f"SELECT * FROM {table_dropdown.value}")
cur.fetchall()
time_slider = IntRangeSlider(value=[0, cur.rowcount], min=0, max=cur.rowcount, step=1200)

def update_time_range(*args):
    cur.execute(f"SELECT * FROM {table_dropdown.value}")
    cur.fetchall()
    time_slider.max = cur.rowcount
table_dropdown.observe(update_time_range, 'value')

def plot_TC(Table_name, Time):    
    cur.execute(f"SELECT * FROM {Table_name}")
    df = pd.DataFrame(cur.fetchall(), columns=[entry[0] for entry in cur.description]).set_index('Id')
    plt.clf()
    #fig, ax1 = plt.subplots()
    #ax2 = ax1.twinx()
    df['TC7'].plot(title=f'TC_{Table_name}', 
                   ylabel='Temp[oC]', 
                   grid=True, 
                   legend=True, 
                   figsize=(20, 10), 
                   secondary_y=False, 
                   colormap='winter', 
                   xlim=Time, 
                   xticks=range(Time[0],Time[1],1200),
                   ylim=(0,1000), 
                   yticks=range(0,1000,50)
                  )
    df['TC8'].plot(legend=True)
    df['TC9'].plot(legend=True)
    df['TC10'].plot(legend=True)
    df['EVA_out'].plot(xlabel='Time[s]', legend=True)
    #ax1.set_ylabel('Temp_SR [oC]', color='g')
    #ax2.set_ylabel('Temp_BR [oC]', color='b')
    plt.show()
    
plot = interactive_output(plot_TC, {'Table_name':table_dropdown, 'Time':time_slider})
box = HBox([table_dropdown, time_slider])
display(box, plot)


Output()

In [ ]:
conn.close()